In [1]:
import wandb
import pandas as pd
import numpy as np
import texttable as tt
import latextable

In [3]:
EXPERIMENT_GROUP = "exp-9-dropout2d"

In [4]:
def get_best_epoch_metrics(run, metric_name, average_epochs=False):
    history = run.history(samples=10000)
    
    # Find the epoch with the best metric value
    best_epoch = history[metric_name].idxmax()
    best_metrics = history.loc[best_epoch]
    
    if average_epochs:
        # Find indices of valid (non-NaN) values
        valid_indices = history[metric_name].dropna().index
        
        # Locate the best epoch index in the valid indices
        best_index_position = valid_indices.get_loc(best_epoch)
        
        # Get the indices of the 5 epochs to be averaged
        start_index = max(0, best_index_position - 2)
        end_index = min(len(valid_indices), best_index_position + 3)
        selected_indices = valid_indices[start_index:end_index]
        
        # Average the metrics over the selected epochs
        averaged_metrics = history.loc[selected_indices].mean()
        
        print(f"Run {run.url} - Best epoch: {best_epoch} (averaged) - {metric_name}: {averaged_metrics[metric_name]}")
        return averaged_metrics, best_epoch
    else:
        print(f"Run {run.url} - Best epoch: {best_epoch} - {metric_name}: {best_metrics[metric_name]}")
        return best_metrics, best_epoch

def gather_metrics_for_experiment_group(experiment_group, average_epochs=False):
    api = wandb.Api()

    # Fetch runs matching the experiment group
    runs = api.runs("crop-classification/messis", filters={"config.experiment_group": experiment_group})
    print(f"Found {len(runs)} runs for experiment group '{experiment_group}'")

    # Organize the runs by setup name
    setups = {}
    for run in runs:
        setup_name = run.config['name']
        if setup_name not in setups:
            setups[setup_name] = {
                'runs': [],
                'metrics': {metric: [] for metric in [
                    'val_f1_tier1_majority',
                    'val_f1_tier2_majority',
                    'val_f1_tier3_majority',
                    'val_weighted_accuracy_tier1_majority',
                    'val_weighted_accuracy_tier2_majority',
                    'val_weighted_accuracy_tier3_majority',
                    'val_precision_tier1_majority',
                    'val_precision_tier2_majority',
                    'val_precision_tier3_majority',
                    'val_recall_tier1_majority',
                    'val_recall_tier2_majority',
                    'val_recall_tier3_majority',
                    'val_cohen_kappa_tier1_majority',
                    'val_cohen_kappa_tier2_majority',
                    'val_cohen_kappa_tier3_majority',
                ]},
                'best_steps': []
            }
        setups[setup_name]['runs'].append(run)

    # Print setup names with number of runs
    for setup_name, setup_data in setups.items():
        print(f"Setup '{setup_name}' - {len(setup_data['runs'])} runs")

    # Gather metrics for each setup
    for setup_name, setup_data in setups.items():
        for run in setup_data['runs']:
            best_metrics, best_epoch = get_best_epoch_metrics(run, 'val_f1_tier3_majority', average_epochs)
            setup_data['best_steps'].append(best_epoch)
            for metric in setup_data['metrics']:
                setup_data['metrics'][metric].append(best_metrics[metric])
    
    # Prepare data for DataFrame
    data = []
    for setup_name, setup_data in setups.items():
        metrics_summary = {
            'setup': setup_name,
            'num_runs': len(setup_data['runs']),
            'run_handles': [run.id for run in setup_data['runs']],
            'best_steps': setup_data['best_steps']
        }
        for metric, values in setup_data['metrics'].items():
            metrics_summary[f'{metric}_mean'] = np.mean(values)
            metrics_summary[f'{metric}_std'] = np.std(values)
        data.append(metrics_summary)
    
    metrics_df = pd.DataFrame(data)
    
    return metrics_df

# Run the function for a given experiment group
metrics_df = gather_metrics_for_experiment_group(EXPERIMENT_GROUP)

# Save the DataFrame to a CSV file
metrics_df.to_csv(f"experiment_results/eval_{EXPERIMENT_GROUP}.csv", index=False)

metrics_df

Found 16 runs for experiment group 'exp-9-dropout2d'
Setup 'p-0.5' - 1 runs
Setup 'p-0.2' - 5 runs
Setup 'refhead-zuericrop' - 5 runs
Setup 'p-0.1' - 5 runs
Run https://wandb.ai/crop-classification/messis/runs/cm0hlw6k - Best epoch: 4858 - val_f1_tier3_majority: 0.1980932652950287
Run https://wandb.ai/crop-classification/messis/runs/rouv9lch - Best epoch: 2446 - val_f1_tier3_majority: 0.18668609857559204
Run https://wandb.ai/crop-classification/messis/runs/ym9requx - Best epoch: 2697 - val_f1_tier3_majority: 0.19291472434997559
Run https://wandb.ai/crop-classification/messis/runs/6hsyjfm2 - Best epoch: 3250 - val_f1_tier3_majority: 0.19807904958724976
Run https://wandb.ai/crop-classification/messis/runs/qhqbesf6 - Best epoch: 3300 - val_f1_tier3_majority: 0.20723950862884521
Run https://wandb.ai/crop-classification/messis/runs/d3zidx38 - Best epoch: 3300 - val_f1_tier3_majority: 0.2096947729587555
Run https://wandb.ai/crop-classification/messis/runs/6vdvcpsa - Best epoch: 2999 - val_f1

,setup,num_runs,run_handles,best_steps,val_f1_tier1_majority_mean,val_f1_tier1_majority_std,val_f1_tier2_majority_mean,val_f1_tier2_majority_std,val_f1_tier3_majority_mean,val_f1_tier3_majority_std,...,val_recall_tier2_majority_mean,val_recall_tier2_majority_std,val_recall_tier3_majority_mean,val_recall_tier3_majority_std,val_cohen_kappa_tier1_majority_mean,val_cohen_kappa_tier1_majority_std,val_cohen_kappa_tier2_majority_mean,val_cohen_kappa_tier2_majority_std,val_cohen_kappa_tier3_majority_mean,val_cohen_kappa_tier3_majority_std
0,p-0.5,1,[cm0hlw6k],[4858],0.361608,0.000000,0.277691,0.000000,0.198093,0.000000,...,0.264429,0.000000,0.186001,0.000000,0.863325,0.000000,0.739953,0.000000,0.701443,0.000000
1,p-0.2,5,"[rouv9lch, ym9requx, 6hsyjfm2, qhqbesf6, d3zid...","[2446, 2697, 3250, 3300, 3300]",0.484903,0.038136,0.314039,0.010112,0.198923,0.008623,...,0.311165,0.008926,0.193074,0.009729,0.866562,0.005090,0.743664,0.005676,0.690875,0.007901
2,refhead-zuericrop,5,"[6vdvcpsa, voz39aqw, tzbb1fpf, yfrzmegx, mhanp...","[2999, 3350, 3099, 2496, 2496]",0.492840,0.058787,0.317704,0.016909,0.204716,0.005751,...,0.319669,0.015285,0.201428,0.008957,0.866212,0.003183,0.743931,0.003949,0.688551,0.011018
3,p-0.1,5,"[lrt47pea, sft8i5l2, hflukrac, il64btbr, 9nyrx...","[3551, 3401, 3953, 2496, 2345]",0.487323,0.050225,0.314840,0.011993,0.205222,0.005220,...,0.315928,0.014720,0.200886,0.006822,0.867367,0.007494,0.745459,0.008520,0.690095,0.014468


In [5]:
df = pd.read_csv(f"experiment_results/eval_{EXPERIMENT_GROUP}.csv")

df = df.sort_values(by='setup')

# Mapping for short metric names
metric_name_mapping = {
    'val_f1_tier1_majority': 'F1 Tier 1',
    'val_f1_tier2_majority': 'F1 Tier 2',
    'val_f1_tier3_majority': 'F1 Tier 3',
    'val_weighted_accuracy_tier1_majority': 'Accuracy Tier 1',
    'val_weighted_accuracy_tier2_majority': 'Accuracy Tier 2',
    'val_weighted_accuracy_tier3_majority': 'Accuracy Tier 3',
    'val_precision_tier1_majority': 'Precision Tier 1',
    'val_precision_tier2_majority': 'Precision Tier 2',
    'val_precision_tier3_majority': 'Precision Tier 3',
    'val_recall_tier1_majority': 'Recall Tier 1',
    'val_recall_tier2_majority': 'Recall Tier 2',
    'val_recall_tier3_majority': 'Recall Tier 3',
    'val_cohen_kappa_tier1_majority': 'Cohen Kappa Tier 1',
    'val_cohen_kappa_tier2_majority': 'Cohen Kappa Tier 2',
    'val_cohen_kappa_tier3_majority': 'Cohen Kappa Tier 3'
}

# Function to create a LaTeX table with bold best scores
def create_latex_table(df):
    metrics = list(metric_name_mapping.keys())
    
    # Determine the best score for each metric
    best_scores = {metric: df[f'{metric}_mean'].max() for metric in metrics}
    
    # Initialize the Texttable object
    table = tt.Texttable()
    
    # Define the header and make headers vertical
    headers = ['Metric'] + [f"{setup} ({row['num_runs']}-fold)" for idx, row in df.iterrows() for setup in [row['setup']]]
    # headers = [f'\\rotatebox{{45}}{{{header}}}' for header in headers]

    table.header(headers)
    
    # Set alignment for columns
    table.set_cols_align(["c"] * len(headers))
    
    # Populate the table with data
    for metric in metrics:
        row_data = [metric_name_mapping[metric]]
        for idx, row in df.iterrows():
            mean = row[f'{metric}_mean'] * 100
            std = row[f'{metric}_std'] * 100
            value = f'{mean:.1f}\\% ± {std:.1f}\\%'
            if mean == best_scores[metric] * 100:
                value = f'\\textbf{{{value}}}'
            row_data.append(value)
        table.add_row(row_data)
    
    # Generate the LaTeX table
    latex_table = latextable.draw_latex(
        table, 
        caption=f"Results for Experiment Group {EXPERIMENT_GROUP}. All metrics calculated on field majority during validation phase, with mean and std reported. Best scores shown in bold.", 
        label=f"tab:metrics_comparison_{EXPERIMENT_GROUP}", 
        use_booktabs=True
    )

    # Add \resizebox to the LaTeX table
    latex_table = latex_table.replace('\\begin{center}', '\\begin{center}\\resizebox{\\textwidth}{!}{')
    latex_table = latex_table.replace('\\end{center}', '}\\end{center}')

    return latex_table

# Generate the LaTeX table
latex_table = create_latex_table(df)
print(latex_table)

\begin{table}
	\begin{center}\resizebox{\textwidth}{!}{
		\begin{tabular}{ccccc}
			\toprule
			Metric & p-0.1 (5-fold) & p-0.2 (5-fold) & p-0.5 (1-fold) & refhead-zuericrop (5-fold) \\
			\midrule
			F1 Tier 1 & 48.7\% ± 5.0\% & 48.5\% ± 3.8\% & 36.2\% ± 0.0\% & \textbf{49.3\% ± 5.9\%} \\
			F1 Tier 2 & 31.5\% ± 1.2\% & 31.4\% ± 1.0\% & 27.8\% ± 0.0\% & \textbf{31.8\% ± 1.7\%} \\
			F1 Tier 3 & \textbf{20.5\% ± 0.5\%} & 19.9\% ± 0.9\% & 19.8\% ± 0.0\% & 20.5\% ± 0.6\% \\
			Accuracy Tier 1 & \textbf{93.3\% ± 0.4\%} & 93.2\% ± 0.2\% & 93.1\% ± 0.0\% & 93.2\% ± 0.2\% \\
			Accuracy Tier 2 & \textbf{82.6\% ± 0.6\%} & 82.4\% ± 0.4\% & 82.2\% ± 0.0\% & 82.4\% ± 0.4\% \\
			Accuracy Tier 3 & 78.0\% ± 1.0\% & 78.0\% ± 0.5\% & \textbf{78.8\% ± 0.0\%} & 77.8\% ± 0.9\% \\
			Precision Tier 1 & 55.7\% ± 5.6\% & \textbf{57.7\% ± 6.4\%} & 45.3\% ± 0.0\% & 56.5\% ± 7.5\% \\
			Precision Tier 2 & 35.1\% ± 1.7\% & 35.6\% ± 1.8\% & 33.0\% ± 0.0\% & \textbf{36.1\% ± 2.7\%} \\
			Precision Tier 3 & 23.3